#### Unusual Datatypes
<table>
<tr>
<th>Variable Name</th>
<th>Current Datatype</th>
<th>Desired Datatype</th>
</tr>
<tr>
<td>HATCH</td>
<td>Object</td>
<td>int64</td>
</tr>
<tr>
<td>SEDAN</td>
<td>Object</td>
<td>int64</td>
</tr>
<tr>
<td>WAGON</td>
<td>Object</td>
<td>int64</td>
</tr>
<tr>
<td>UTE</td>
<td>Object</td>
<td>int64</td>
</tr>
<tr>
<td>K_SALES_TOT </td>
<td>Object</td>
<td>int64</td>
</tr>
</table>
     
For the given data description, the fields `UTE`, `HATCH`,`SEDAN`, `WAG0N` and `K_SALES_TOT` should be interval/numerical values as opposed to objects.
By using the `.describe()` function, we may be able to uncover the source of the issues in the dataset

# IFN645 Case Study 2
## Mining from Manufacturing, Supermarket, News Stories and Web Log Data

### Contents
1. [Clustering & Pre-processing](#clust)
2. [Association Mining](#association)
3. [Text Mining](#text)
4. [Web Mining](#web)

---
## Part 1: Clustering Pre-processing and K-means analysis<a name="clust"></a>

In [ ]:
# Libraries Required for this section
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Import Data for this section from csv (without skipping empty cells)
df = pd.read_csv('Casestudy2-Data-Py/model_car_sales.csv', na_filter=False)

# Get details about the dataset 
df.info()

##### Description of Dataset
Below is a description of the data as it appears in the brief
<table>
<tr>
<th>Variable</th>
<th>Measurement Level</th>
<th>Description</th>
</tr>
<tr>
<td>LOCATION_NUMBER</td>
<td>Nominal</td>
<td>Numeric code for the store</td>
</tr>
<tr>
<td>REPORT_DATE</td>
<td>Unary</td>
<td>Date of the data extraction</td>
</tr>
<tr>
<td>DEALER_CODE</td>
<td>Nominal</td>
<td>Text identifier for the store</td>
</tr>
<tr>
<td>UTE</td>
<td>Interval</td>
<td>Number of Utility/tray back model cars sold by the store</td>
</tr>
<tr>
<td>HATCH</td>
<td>Interval</td>
<td>Number of Utility/tray back model cars sold by the store</td>
</tr>
<tr>
<td>WAG0N</td>
<td>Interval</td>
<td>Number of Station Wagon model cars sold by the store</td>
</tr>
<tr>
<td>SEDAN</td>
<td>Interval</td>
<td>Number of Sedan model cars sold by the store</td>
</tr>
<tr>
<td>K__SALES_TOT</td>
<td>Interval</td>
<td>Total sales for the store</td>
</tr>
</table>

### 1. Can you identify data quality issues in this dataset such as unusual data types, missing values, etc?
By comparing the output from the `info()` method to the table above, a number of data quality issues emerge.

The rows `HATCH`,`SEDAN`,`WAG0N`,`UTE` and `K__SALES_TOT`are of the type `object` in the dataframe, where the data description suggests that the values should be of the type `int` or `float`.

Using the `.describe()` function may uncover the root of the problem within the data set.

In [ ]:
# print details for all variables in dataframe
def describeDataset(doc):
    """
    Print Details for each column
    """
    for cols in doc:
        print(df[cols].describe())
        print("-"*20)
        
describeDataset(df)

Analysis of the `.describe()` output show a number of unusual results which may explain the cause of the mismatched data types.

For this output, `top` describes the most commonly occuring value in the dataset. For the fields `UTE`, `HATCH`, `WAG0N` and `SEDAN`, that value appears to be an empty string.

Using the `unique()` function (see below) allows us to identify this as the root cause of our data abnormalities.

In [ ]:
# Check for cause of issues in one of the variables
print(df['UTE'].unique())

Additionally, we can isolate the rows containing the empty string in order to determine if there is a link between the missing variables in the data source.

By using the `.values()` or `as_matrix()` functions to gather all of the readings where UTE is equal to an empty string we get the following output:

In [ ]:
# See Rows where UTE, HATCH, WAG0N or SEDAN contains empty string
mat_view = df[(df['UTE']=='')|(df['HATCH']=='')|(df['WAG0N']=='')|(df['SEDAN']=='')].as_matrix()
print(mat_view)
print("Num Elements: ",len(mat_view))

#### Missing Values
As seen above, 22 rows in the dataset contain empty string values for `UTE`, `HATCH`,`WAG0N` or `SEDAN`. This additionally identifies 22 rows where the `K_SALES_TOT` variable is also missing. A tabulated view of the results can be found below.

<table>
    <tr>
        <th>Variable Name</th>
        <th># Missing Values</th>
    </tr>
    <tr>
        <td>HATCH</td>
        <td>22</td>
    </tr>
    <tr>
        <td>SEDAN</td>
        <td>22</td>
    </tr>
    <tr>
        <td>WAGON</td>
        <td>22</td>
    </tr>
    <tr>
        <td>UTE</td>
        <td>22</td>
    </tr>
    <tr>
        <td>K_SALES_TOT</td>
        <td>22</td>
    </tr>
</table>

For the purposes of performing KMeans clustering on this data, these rows can be dropped from the dataset using the method below.

In [ ]:
# Clean and return dataset
def cleanAndConvertDataset1(doc):
    # replace empty elements with np.nan and cast fields to float
    return df[['HATCH','SEDAN','WAG0N','UTE','K__SALES_TOT']].replace('',np.nan).astype(float)
df = cleanAndConvertDataset1(df)
df.info()

Finally, visualisation of the data can assist in identifying any remaining data problems.
Using `seaborn` and `matplotlib`; we can graph the distributions for `UTE`, `HATCH`,`WAGON` and `SEDAN` in the dataset.

In [ ]:
# Distribution of UTE
ute_dist = sns.distplot(df['UTE'].dropna())
plt.show()

# Distribution of Hatch
hatch_dist = sns.distplot(df['HATCH'].dropna())
plt.show()

# Distribution of Wagon
wagon_dist = sns.distplot(df['WAG0N'].dropna())
plt.show()

# Distribution of Sedan
sedan_dist = sns.distplot(df['SEDAN'].dropna())
plt.show()

As we can see in the distplots, there are a number of anomalies for the distribution of `WAG0N` in the dataset - particularly for wagon sales below 200 units. 

In order to quanitfy the anomalous values, we can increasae the number of bins and isolate the data to sales below 200 units as seen below

In [ ]:
# Distribution of Wagon
wagon_dist = sns.distplot(df[df['WAG0N'] < 200]['WAG0N'].dropna(),bins=100)
plt.show()

# Print rows with the bottom 10 sales for the station wagons
print(df.sort_values(by='WAG0N')[df['WAG0N'] < 200].head(20))

#### Outliers
As shown above, the distplot for stores selling fewer than 200 station wagons shows an unusually high number of stores selling fewer than 50 station wagons.

Additionally, by comparing sales of Station Wagon against sales of other car models in these instances, We can identify a number of cases where the sales of Station Wagons are over 10 times lower than the nearest model at the same dealership. This pattern appears to be indicative of erronious data. As K-Means clustering is sensitive to outliers, these rows should be dropped from the analysis.

In [ ]:
# Drop Outliers from WAG0N
df1 = df[df['WAG0N']>30]

# Distribution of Wagon
wagon_dist = sns.distplot(df['WAG0N'].dropna())
wagon_dist = sns.distplot(df1['WAG0N'].dropna())
plt.show()

### 2. What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice.
For the purposes of clustering, we will include the `HATCH`, `SEDAN`, `WAG0N` and `UTE` variables.
These variables measure the number of sales of a particular model of vehicle made by particular a car dealership (signified by `LOCATION_NUMBER` or `DEALER_CODE`).

The variables `LOCATION_NUMBER` and `DEALER_CODE` will not be included in the analysis, as unique variables do not contribute to clustering models. Additionally, `REPORT_DATE` will not be included in the analysis as it contains the same value for all observations. Finally, `K_SALES_TOT` will also be discluded from the Dataset, as the derrived value is not useful for our analysis.

<table>
<tr>
        <th>Variable</th>
        <th>Role</th>
        <th>Measurement Level Set</th>
</tr>
<tr>
        <td>LOCATION_NUMBER</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>REPORT_DATE</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>DEALER_CODE</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>HATCH</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>WAG0N</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>SEDAN</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>UTE</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
<tr>
        <td>K__SALES_TOT</td>
        <td>Input</td>
        <td>Interval</td>
</tr>
</table>

In [ ]:
df2 = df1[['UTE','HATCH','WAG0N','SEDAN']].dropna()
df2.info()

### 3. Identify a car model that is underperforming in sales. Based on your reporting, the company does not want to focus thir efforts on this car model anymore and has decided to drop it from manufacturing. Now onwards, the selected car product should not be part of analysis
By comparing the distributions of each model of car, it is possible to identify how car sales are distributed among dealerships.

Below are the sales distributions of each car model.

In [ ]:
# Distribution of all Models
comb_dist = sns.distplot(df2['UTE'].dropna(),kde_kws={"label":"UTE"})
comb_dist = sns.distplot(df2['HATCH'].dropna(),kde_kws={"label":"HATCH"})
comb_dist = sns.distplot(df2['WAG0N'].dropna(),kde_kws={"label":"WAG0N"})
comb_dist = sns.distplot(df2['SEDAN'].dropna(),kde_kws={"label":"SEDAN"})
plt.show()

As shown above, the `UTE` model has the poorest performance among vehicle models, with the majority of dealerships selling fewer than 100 models over the sales period.

Due to this, the `UTE` model will be removed from the analysis as instructed by the brief.

In [ ]:
# Drop underperforming vehicle UTE
df2.drop(['UTE'],axis=1,inplace=True)
df2.info()

## Task 2. The First Clustering Model
Below is the method that will be used to perform clustering in this section.

In [ ]:
def performKMeansClustering(docIn, k=3, scale=False,rs=42, report_cluster_distances=True):
    """
    Perform KMeans clustering on provided dataset and assign Model ID to each row
    """
    # Create copy of input dataset to return
    doc = docIn.copy()
    
    # convert the dataset into a matrix
    X = doc.as_matrix()
    
    if scale:
        # scale the variables
        s = StandardScaler()
        X = s.fit_transform(X)

    # set the random state for the model
    model = KMeans(n_clusters=k, random_state=rs).fit(X)

    if report_cluster_distances:
        # report sum of intra-cluster distances
        print("Sum of intra-cluster distance:", model.inertia_)
        print("Centroid Locations:")
        for c in model.cluster_centers_:print(c)
            
    # assign cluster to each record in X
    y = model.predict(X)
    
    # Assign cluster id's to each row and return 
    doc['Cluster_ID'] = y
    
    return doc, X

### 1. Build a default clustering model with K = 3
Using the method above, clustering is performed on the dataset using the default K of 3

In [ ]:
# Perform KMeans clustering on dataset without normalisation
K3_NoScale, X_NoScale = performKMeansClustering(df2)

In [ ]:
def reportClusterMembership(doc):
    # print the number of members for each cluster
    print("Distribution of Cluster Members")
    print(doc['Cluster_ID'].value_counts())
    
reportClusterMembership(K3_NoScale)

#### a. How many records are assigned into each cluster?
As seen in the output of the `reportClusterMembership()` method above, the cluster membership is as follows:

<table>
<tr>
<th>Cluster ID</th>
<th>Number of members</th>
</tr>
<tr>
<td>0</td>
<td>264</td>
</tr>
<tr>
<td>1</td>
<td>189</td>
</tr>
<tr>
<td>2</td>
<td>167</td>
</tr>
</table>

#### b. Plot the cluster distribution using pairplot. Explain key characteristics of each cluster/segment
The key characteristics of each cluster are as follows

In [ ]:
def visCluster(doc):
    """
    Generate Pair Plot to visualise clusters
    """
    # Generate Pairplot for clusters
    cluster = sns.pairplot(doc, hue='Cluster_ID')
    plt.show()

# Plot clusters as pairplot
visCluster(K3_NoScale)

In [ ]:
def clusterDistributions(doc, n_bins=20, cols=[], clusters=[]):
    """
    Plot distribution of variables in each cluster, in relation to distributions of varibles in the dataset
    """
    
    # if specific clusters not provided, go over entire range
    if len(clusters) == 0:
        clusters_to_inspect = doc['Cluster_ID'].unique()

    for cluster in clusters_to_inspect:
        # inspecting cluster 0
        print("Distribution for cluster {}".format(cluster))

        # create subplots
        fig, ax = plt.subplots(nrows=3)
        ax[0].set_title("Cluster {}".format(cluster))
        
        

        for j, col in enumerate(cols):
            # create the bins
            bins = np.linspace(min(doc[col]), max(doc[col]), 20)
            # plot distribution of the cluster using histogram
            sns.distplot(doc[doc['Cluster_ID'] == cluster][col], bins=bins, ax=ax[j], norm_hist=True)
            # plot the normal distribution with a black line
            sns.distplot(doc[col], bins=bins, ax=ax[j], hist=False, color="k")

        plt.tight_layout()
        plt.show()
        
# prepare the column and bin size. Increase bin size to be more specific, but 20 is more than enough
cols = ['HATCH', 'WAG0N', 'SEDAN']
clusterDistributions(K3_NoScale,cols=cols)

#### Cluster 0
As seen in the distributions for cluster 0:
Dealerships in cluster 0 sell an average number of all vehicles. Due to the volume of sales for each vehicle, this can be interpreted as a predominance in sales of Hatch and Sedan models 

#### Cluster 1
As seen in the distributions for cluster 1:
Dealerships in cluster 1 sell the mostly Hatch Back models and average to low numbers of Station Wagons and Sedans respectively. Due to the volume of sales, this could be interpreted as selling a majority of Hatch Back models with fewer sales of Sedan models.

#### Cluster 2
As seen in the distributions for cluster 2:
Dealerships in cluster 2 sell an above average number of station wagon and sedan model cars, with below average sales of Hatch Back model cars. 

These characteristics are corroborated by the distributions per cluster, found below.

### 2. What is the effecti of using the standardization method on the model above? Does the variable normalization process enable a better clustering solution?
The process above was repeated using the `StandardScalar` modlue from the `SKlearn` library

In [ ]:
# Perform K means clustering with normalised variables
K3_Scaled, X_Scaled = performKMeansClustering(df2,scale=True)

In [ ]:
# Plot clusters for the normalised dataset
visCluster(K3_Scaled)

In [ ]:
# View Distributions of clusters in the normalised dataset
clusterDistributions(K3_Scaled,cols=cols)

As seen in the plots above, standardisation results in more cohesive clustering. As cluster centroids in K Means analysis are based on euclidean distances, Standardising the scales of values involved in the analysis allow variables with different ranges to be effectively compared.

### 3. Interpret the cluster analysis outcome. In other words, characterize the nature of each cluster by giving it a descriptive label

Cluster 0 - Sedan Dominant Dealerships
Dealerships belonging to Cluster 0 typically make above average sales on Sedan Models and average to below average sales on Station Wagon and Hatch Back Models

Cluster 1 - Hatch Dominant Dealerships
Dealershis belonging to Cluster 1 sell an above number of Hatch Back models and make average to below average sales of Station Wagon and Sedan Models Respectively

Cluster 2 - Wagon & Sedan Dominant Dealerships
Dealerships belonging to Cluster 2 make above average sales of Wagon and Sedan models, with below average sales of Hatch models.

## Task 3. Refining the clustering model
### 1. Using elbow method and silhouette, find the optimal K. What is teh best K? Explain your reasoning. Evaluate the results
Using the `computeKGraph` method below, the model inertia for each value of K between 2 and 8 can be graphed to determine the 'elbow'

In [ ]:
def computeKGraph(doc,X,rs=42,k_range=range(2,9,1)):
    """
    Plot cluster model inertia for given dataset with different leves of K applied
    """
    # Lists for clusters and costs
    clusters = []
    inertia_vals = []

    # calculate inertia for k
    for k in k_range:
        # train with the current K value
        model = KMeans(n_clusters=k, random_state=rs, n_jobs=10)
        model.fit(X)

        # append model to lits
        clusters.append(model)
        inertia_vals.append(model.inertia_)

    # Plot the outpt of inertai vs the value of K
    plt.plot(range(2,9,1), inertia_vals, marker='*')
    plt.show()
    return clusters, inertia_vals
    
# Perform analysis on previously calculated dataset
clusters, inertia = computeKGraph(K3_Scaled,X_Scaled)

### 2. What is the best number of clusters that can describe the dataset effectively? How was this obtained?
Using the `identifyOptimalK` method below, we can determine the optimal value of K for a given dataset by identifying the model with the highest silhouette score.

In [ ]:
def identifyOptimalK(evalclusters, X_in,verbose=False):
    """
    Use the elbow rule to identify the optimal value of K
    """
    # Initialise variables for storing optimal K value
    best_K = 0
    maxSilhouette = 0
    
    # Iterate through all elements in 
    for clust in evalclusters:
        # Get silhouette score for cluster
        sil = silhouette_score(X_in, clust.predict(X_in))
        if sil > maxSilhouette:
            maxSilhouette = sil
            best_K = clust.n_clusters
        
        # Print verbose details on each cluster
        if verbose:
            print(clust)
            print("Silhouette Score for K={}".format(clust.n_clusters), sil)
            print("-"*20)
    
    # Print results and return K
    print("Optimal K = ",best_K, "with score ", maxSilhouette)
    return best_K
        
# Compute optimal K based on the provided dataset
optimal_K = identifyOptimalK(clusters,X_Scaled)

As seen in the output above, the optimal value of K for the dataset was determined to be 4. This results in the following clusters.

In [ ]:
# Perform KMeans clustering on dataset without normalisation
K_Optimal, X_Optimal = performKMeansClustering(df2,k=optimal_K,scale=True)

# Plot clusters for the normalised dataset
visCluster(K_Optimal)

### 3. How can the outcome of this study be used by decision makers?
Clusters allow decision makers to identify trends in the sales of vehicle models for each dealership.

This information can be used to optimise marketing material to cater specifically to the predominent modles sold by dealerships belonging to a cluster.

---
## Part 2: Association Mining and it's data Pre-processing<a name="association"></a>

### 1. Can you identify data quality issues in this dataset for performing association mining?


In [ ]:
import pandas as pd

# load the transaction dataset
df = pd.read_csv('Casestudy2-Data-Py/pos_transactions.csv')

# info and the first 10 transactions
print(df.info())
print(df.head(10))

From the first 10 transaction shown above, we can easily see that there are duplicated transactions in the dataset.
That can be seen clearly by grouping the data (see the result below)

In [ ]:
df.groupby(df.columns.tolist(),as_index=False).size().sort_values(ascending = False).reset_index().rename(columns={0: 'Frequency'})

The 'Frequency' column in the result above indicates the number of duplicated transactions for each unique transaction in the dataset. Therefore, we will drop those duplicated transactions except for the first occurence. Hence the number of transactions will be decreased that help the apriori algorithm (in the question 3)run faster 

In [ ]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)

As were are looking to generate association rules from items purchased by each transaction, we need to group our Transaction_Id and then generate a list of all items purchased.

In [ ]:
# group by Transaction_Id, then list all items
transactions = df.groupby(['Transaction_Id'])['Product_Name'].apply(list)

print(transactions.head(10))

### 2. What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice.
Association mining usually consists of two variables: a transaction ID and an item. Due to the main target which is to find out the associations between items purchased from the health and beauty aids department and the stationary department, the Product_Name should be chosen as the target variable and the Transaction_ID is chosen as ID. In summary, the variables included in the analysis as the below table.

<table>
    <tr>
        <th>Variable Name</th>
        <th>Role</th>
        <th>Measurement Level</th>
    </tr>
    <tr>
        <td>Product_Name</td>
        <td>Target</td>
        <td>Nominal</td>
    </tr>
    <tr>
        <td>Transaction_Id</td>
        <td>ID</td>
        <td>Nominal</td>
    </tr>
</table>

### 3. Conduct association mining and answer the following:
#### a. What is the highest lift value for the resulting rules? Which rule has this value?

Once the transactions table contains all items purchased in each transaction, we will run the apyori model with the pre-processed transactions and min_support of 0.01. This min support is chosen because the number of rules decreases when the min support increases (see the table below) 
<table>
    <tr>
        <th>Min Support</th>
        <th>Number of rules</th>
    </tr>
    <tr>
        <td>0.01</td>
        <td>72</td>
    </tr>
    <tr>
        <td>0.02</td>
        <td>38</td>
    </tr>
    <tr>
        <td>0.03</td>
        <td>29</td>
    </tr>
</table>

In [ ]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.01))

# print first 5 rules
print(results[:5])

In [ ]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(len(result_df))
print(result_df.head(20))

In order to find out the highest lift value for the resulting rules, we will sort the rules by Lift using the code below:

In [ ]:
# sort all acquired rules descending by lift# sort a 
result_df = result_df.sort_values(by='Lift', ascending=False)
print(len(result_df))
print(result_df.head(20))


From the result above, the highest lift value is 3.60
There are two rules having this value: Perfume -> Toothbrush and Toothbrush -> Perfume

#### b. What is the highest confidence value for the resulting rules? Which rule has this value?

In order to find out the highest confidence value for the resulting rules, we will sort the rules by Confidence using the code below:

In [ ]:
# sort all acquired rules descending by Confidence# sort a 
result_df = result_df.sort_values(by='Confidence', ascending=False)
print(result_df.head(20))

From the result above, the highest confidence value is 46.37%.
The rule has this value is Magazie & Greeding Cards -> Candy Bar  

#### c. Plot the confidence, lift, support of the resulting rules? Interpret them to discuss the rule-set obtained

In [ ]:
support = result_df['Support'].as_matrix()
confidence = result_df['Confidence'].as_matrix()
lift = result_df['Lift'].as_matrix()
lift

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(3, figsize=(7,5))
plt.scatter(support, confidence, c=lift, cmap='Reds')
plt.xlabel('support')
plt.ylabel('confidence') 

plt.colorbar().set_label('Lift')


The figure above shows describes the support, confidence and lift for rules discovered during the association mining step.

Support describes the probability of a rule appearing within the dataset. With respects to the data, Support describes the frequency with which Itemset A appears with Itemset B in the data.
`Frequency of A and B/Total number of Rows`

Confidence of rule A=>B describes the probability of A and B occuring together in all elements containing A. With respects to the data, Confidence measures how frequently item A is purchased with item B against the frequency with which only A is present.
`Frequency of A and B/Frequency of only A`

Support and Confidence are important metrics for discovering the strength of a given association rule, however, high confidence is not necessarily indicative of a useful rule. Lift describes a ratio of confidence against support; This represents the 'predictive power' of a rule compared to random chance.
For example, a lift of 3 for A=>B indicates that customers who purchase A are 3 times more likely to also purchase B

- As we can see from the figure above, the rules having lift values more than 2.0 have high confidence and low support that is a good indicator. 
- The rules having low lift values have low support that is not a good indicator



### 4. The store is particularly interested in products that individuals purchase when they buy “Pens”.
#### a. How many rules are in the subset?

In [ ]:
result_df.loc[result_df['Left_side'] == 'Pens']

There is 3 rules are in the subset

#### b. Based on the rules, what are the other products these individuals are most likely to purchase?

In [ ]:
pensDf = result_df[result_df['Left_side'] == 'Pens']
otherProducts = pensDf.groupby(['Left_side'])['Right_side'].apply(list)
otherProducts.tolist()

The other products these individuals are most like ly to purchase are Magazine, Candy Bar, Toothpaste

### 5. How the outcome of this study can be used by decision makers?

Based on the interest of the sore in determining the associations between items purchased from the health and beauty aids department and the stationary deparment, it should be benefit to consider the folowwing rules:

Toothpaste,Pencils  =>  Candy Bar

Magazine,Greeting Cards  =>  Candy Bar

Toothpaste,Magazine  =>  Candy Bar

Candy Bar,Magazine  =>  Greeting Cards

Candy Bar,Greeting Cards  =>  Magazine

Toothpaste,Greeting Cards  =>  Candy Bar

Pencils,Candy Bar  =>  Magazine

Pencils,Magazine  =>  Greeting Cards

Candy Bar,Magazine  =>  Toothpaste

Magazine,Greeting Cards  =>  Pencils

Toothbrush  =>  Perfume

The decision makers can use the rules extracted from this association mining to obtain the understainding of consumers' needs and behaviours. Based on these rules, several marketing strategies can be made, for example:

- The association rules can tell decision makers that there are several items that are often bought together. Therefore, promoting the items together can help increase the revenue. For example, promoting the combination of Candy Bar & Greeting Cards, Toothpase & Pencils, etc. 
- These rules can also be used to structure supermarket layout. For example, putting Gretting Cards on the shelves next to the Candy Bar shelves
- Based on the outcome of association mining, the purchasing pattern can be derrived. From there, customer segments can be identified
- Additionally, the outcome can be used to design catologies for the supermarket

---
## Part 3: Text Mining<a name="text"></a>

### 1.	What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice.

The selected column is the TEXT column, as this column contains the text data from which information will be extracted. All other columns are rejected.

The LANGUANGE, OMITTED, TRUNCATED, and EXTENSION columns are unary, and provide no meaningful data. Similarly, the URI, NAME, and FILTERED columns are incremental values that also provide no meaningful data. Finally, the columns CREATED, ACCESSED, and MODIFIED, appear to be date based values. Modified is binary, and the other columns only have thirty five unique values.

In [ ]:
import pandas as pd

tf = pd.read_csv("Casestudy2-Data-Py/bbc.csv")

checkCols = ["CREATED", "ACCESSED", "MODIFIED"]

for col in checkCols:
    print(col, "- Unique:", len(tf[col].unique()))

### 2. Can you identify data quality issues in order to perform text mining?

In order to prepare the text for mining, it was first stripped of punctuation and analytically useless stopwords obtained from nltk StopWords. Secondly, the text is lemmatised using the nltk WordNet dictionary, in order to reduce words to their base format. Finally, the text is vectorised using the sklearn tf idf vectoriser, so that it can be used for clustering.

In [ ]:
import string

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

# initialise WordNet lemmatizer and punctuation filter
lemmatizer = WordNetLemmatizer()
punct = set(string.punctuation)

stopwords = set(sw.words('english'))
stopwords.update('�')
textColumn = 'TEXT'
clusters = 7
rs = 42
docTotal = len(tf[textColumn]) #store total documents for calculating document frequency percentages

In [ ]:
def lemmatize(token, tag):
    tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
        'R': wn.ADV,
        'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    return lemmatizer.lemmatize(token, tag)

def cab_tokenizer(document):
    # initialize token list
    tokens = []
    
    # split the document into sentences
    for sent in sent_tokenize(document):
        # split the document into tokens and then create part of speech tag for each token
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # preprocess and remove unnecessary characters
            token = token.lower()
            token = token.strip()
            token = token.strip('_')
            token = token.strip('*')

            # If stopword, ignore token and continue
            if token in stopwords:
                continue

            # If punctuation, ignore token and continue
            if all(char in punct for char in token):
                continue

            # Lemmatize the token and add back to the tokens list
            lemma = lemmatize(token, tag)
            tokens.append(lemma)
    
    return tokens

from sklearn.feature_extraction.text import TfidfVectorizer

# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2))
X = tfidf_vec.fit_transform(tf[textColumn])
weights = tfidf_vec.idf_

# see the number of unique tokens produced by the vectorizer. Lots of them...
print("Unique tokens:", len(tfidf_vec.get_feature_names()))

Without document and term frequency filtering, there are 36340 unique tokens found by the tf idf vectorizer. This is far too many to generate meaningful clusters.

In [ ]:
from sklearn.cluster import KMeans

# K means clustering using the term vector
kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)

In [ ]:
# function to visualise text cluster. Useful for the assignment too :)
def visualise_text_cluster(n_clusters, cluster_centers, terms, num_word = 7):
    # -- Params --
    # cluster_centers: cluster centers of fitted/trained KMeans/other centroid-based clustering
    # terms: terms used for clustering
    # num_word: number of terms to show per cluster. Change as you please.
    
    # find features/terms closest to centroids
    ordered_centroids = cluster_centers.argsort()[:, ::-1]
    
    for cluster in range(n_clusters):
        print("Top terms for cluster {}:".format(cluster), end=" ")
        for term_idx in ordered_centroids[cluster, :num_word]:
            print(terms[term_idx], end=', ')
        print()
        
# call it
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

As stated above, the clusters generated with unfiltered terms are damaged by high frequency, low value terms, such as terms like 'say', 'play', and 'ball'. These terms are vague and do not allow for meaningful conclusions to be drawn from the clusters. As such, the terms must be filtered using term/document frequency.

### 3. Based on the ZIPF plot, list the top 10 terms that will be least useful for clustering purpose.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# creating tf-idf terms - a bit slow, do it occasionaly
def calculate_tf_idf_terms(document_col):
    # Param - document_col: collection of raw document text that you want to analyse
    from sklearn.feature_extraction.text import CountVectorizer

    # use count vectorizer to find TF and DF of each term
    count_vec = CountVectorizer(tokenizer=cab_tokenizer, ngram_range=(1,2))
    X_count = count_vec.fit_transform(tf[textColumn])
    
    # create list of terms and their tf and df
    terms = [{'term': t, 'idx': count_vec.vocabulary_[t],
              'tf': X_count[:, count_vec.vocabulary_[t]].sum(),
              'df': X_count[:, count_vec.vocabulary_[t]].count_nonzero(),
              'weights': weights[i]}
             for i, t in enumerate(count_vec.vocabulary_)]
    
    return terms
    
terms = calculate_tf_idf_terms(tf[textColumn])

In [ ]:
# visualisation of ZIPF law
def visualise_zipf(terms, itr_step = 50):
    from scipy.spatial.distance import euclidean
    from math import sqrt
    
    # --- Param ---
    # terms: collection of terms dictionary from calculate_tf_idf_terms function
    # itr_step: used to control how many terms that you want to plot. Num of terms to plot = N terms / itr_step
    
    # sort terms by its frequency
    terms.sort(key=lambda x: (x['tf'], x['df']), reverse=True)
    
    # select a few of the terms for plotting purpose
    sel_terms = [terms[i] for i in range(0, len(terms), itr_step)]
    labels = [term['term'] for term in sel_terms]
    
    # plot term frequency ranking vs its DF
    plt.plot(range(len(sel_terms)), [x['df'] for x in sel_terms])
    plt.xlabel('Term frequency ranking')
    plt.ylabel('Document frequency')
    
    max_x = len(sel_terms)
    max_y = max([x['df'] for x in sel_terms])
    
    # annotate the points
    prev_x, prev_y = 0, 0
    for label, x, y in zip(labels,range(len(sel_terms)), [x['df'] for x in sel_terms]):
        # calculate the relative distance between labels to increase visibility
        x_dist = (abs(x - prev_x) / float(max_x)) ** 2
        y_dist = (abs(y - prev_y) / float(max_y)) ** 2
        scaled_dist = sqrt(x_dist + y_dist)
        
        if (scaled_dist > 0.1):
            plt.text(x+2, y+2, label, {'ha': 'left', 'va': 'bottom'}, rotation=30)
            prev_x, prev_y = x, y
    
    plt.show()
    
    return terms

#Store terms ordered by term frequency, document frequency
orderedTerms = visualise_zipf(terms).copy()

#Reorder terms by weights
terms.sort(key=lambda x: (x['weights']), reverse=True)
for i, term in enumerate(terms[0:10]):
    print(str(i + 1) + ")\tTerm:", term['term'], "\tFrequency:", term['tf'], "\tWeight:", term['weights'])

Using ZIPF law, the least useful terms can be calculated. As visualised above, the Top 10 least useful terms by weight are:

1. say
2. win
3. one
4. go
5. two
6. time
7. first
8. get
9. player
10. make

These terms have the highest tf-idf weight values, and thus provide no meaningful value to the clustering process.

### 4. Did you disregard any frequent terms? Justify their selection.

Examining the top 25 most frequent terms from the dataset, it can be observed that there is a large amount of vague sports terminology. All of the top ten least useful terms can be observed in the top 25 most frequent. The most frequent term, 'say', also provides no value.

In [ ]:
#An array of zeroes equal to the number of documents, to count document frequencies
dfCount = [0] * docTotal

for i, term in enumerate(orderedTerms):
    if i < 25:
        print(str(i + 1) + ")\tTerm:", term['term'], "\tDocument Frequency:", (term['df'] / docTotal * 100), "%\tTerm Frequency", term['tf'])
    dfCount[term['df'] - 1] += 1

Most of the high frequency terms occur within 40-50% of the documents, except for the 9th term, 'england'. This term can be useful for clustering, and may provide useful information. All terms above 40% document frequency (appearing in over 80 documents), will be disregarded through filtering.

In [ ]:
# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2), min_df=1, max_df=0.4)
X = tfidf_vec.fit_transform(tf[textColumn])

# see the number of unique tokens produced by the vectorizer. Lots of them...
tokenCount = len(tfidf_vec.get_feature_names())
print("Unique tokens for 40%:", tokenCount)

kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

Limiting the maximum document frequency to 40% removes 20 terms, which is terms 1-21, excluding 'england'.

### 5. Justify the term weighting option selected

The maximum document frequency is set to 40% to remove vague, high frequency terms, as explained above.

In [ ]:
#Show distribution of document frequencies.
plt.bar(range(1, len(dfCount) + 1), dfCount)
plt.show()

print("Document Frequency 1 - 5:", sum(dfCount[0:5]), "Remainder:", tokenCount - sum(dfCount[0:5]),  "Elements:", dfCount[0:5])

There are 35101 tokens that only appear in 5 or less documents, and over 29435 only appear in 1. Due to this large portion, filtering a higher minimum document frequency may remove too many terms.

In [ ]:
# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2), min_df=2, max_df=0.4)
X = tfidf_vec.fit_transform(tf[textColumn])

# see the number of unique tokens produced by the vectorizer. Lots of them...
tokenCount = len(tfidf_vec.get_feature_names())
print("Unique tokens for min_df = 2:", tokenCount)

kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

print()
# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.4)
X = tfidf_vec.fit_transform(tf[textColumn])

# see the number of unique tokens produced by the vectorizer. Lots of them...
tokenCount = len(tfidf_vec.get_feature_names())
print("Unique tokens for min_df = 3:", tokenCount)

kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

print()
# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2), min_df=4, max_df=0.4)
X = tfidf_vec.fit_transform(tf[textColumn])

# see the number of unique tokens produced by the vectorizer. Lots of them...
tokenCount = len(tfidf_vec.get_feature_names())
print("Unique tokens for min_df = 4:", tokenCount)

kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

print()
# tf idf vectoriser
tfidf_vec = TfidfVectorizer(strip_accents = 'unicode', tokenizer=cab_tokenizer, ngram_range=(1,2), min_df=5, max_df=0.4)
X = tfidf_vec.fit_transform(tf[textColumn])

# see the number of unique tokens produced by the vectorizer. Lots of them...
tokenCount = len(tfidf_vec.get_feature_names())
print("Unique tokens for min_df = 5:", tokenCount)

kmeans = KMeans(n_clusters=clusters, random_state=rs).fit(X)
visualise_text_cluster(kmeans.n_clusters, kmeans.cluster_centers_, tfidf_vec.get_feature_names())

Examining the clusters generated at different min_df levels, setting the minimum document frequency to 4 produces the best results. Meaningful clusters are generated such as the cricket tests for cluster 0, and athletics bans for cluster 4.

### 6. What is the number of input features available to execute clustering?  (For information: Note how the original text data is now converted into a feature set that can be mined for knowledge discovery?)

To reduce the tfidf frequency matrix, Singular Value Decomposition was used. By setting the number of components to 100, a significantly smaller matrix is created.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, random_state=rs)
X_trans = svd.fit_transform(X)

# sort the components by largest weighted word
sorted_comp = svd.components_.argsort()[:, ::-1]
terms = tfidf_vec.get_feature_names()

# visualise word - concept/component relationships
for comp_num in range(10):
    print("Top terms in component #{}".format(comp_num), end=" ")
    for i in sorted_comp[comp_num, :5]:
        print(terms[i], end=", ")
    print()

In [ ]:
# K-means clustering using LSA-transformed X
svd_kmeans = KMeans(n_clusters=7, random_state=rs).fit(X_trans)

# transform cluster centers back to original feature space for visualisation
original_space_centroids = svd.inverse_transform(svd_kmeans.cluster_centers_)

# visualisation
visualise_text_cluster(svd_kmeans.n_clusters, original_space_centroids, tfidf_vec.get_feature_names(), num_word = 15)

### 7. State how many clusters are generated? Name each cluster meaningfully according to the terms that appear in the clusters?

There were seven clusters generated through singular value decomposition. These clusters each had meaningful categories and themes.

Cluster 1: Australian Open Tennis

Containing terms such as Hewitt, Federer, Australian Open and Grand Slam, this cluster is about professional tennis.

Cluster 2: Six Nation Rugby Union
      
Featuring the term 'Six Nation' and the six nations that competed in the tournament, 'Ireland', 'England', 'Wales', 'Scotland', 'Italy' and 'France', this cluster focuses on the annual Six Nationa Rugby Union competition     
        
Cluster 3: Rugby Team Changes

This cluster is about changes in staff and players in rugby teams. This is evidenced by the terms 'rugby', 'club', 'director', 'coach', 'new' and 'want'

Cluster 4: Athletics Drug Bans

With the terms 'Athens', 'Olympic', 'athletics', 'drug' and 'ban', this cluster is about drug related bans occurring at the Athens Olympics.

Cluster 5: Cricket Tests

Featuring cricketing nations such as 'Pakistan', 'Australia', 'South Africa' and 'India', as well as the terms 'cricket', 'test', 'wicket' and 'run'. This section is about cricket test matches between nations.

Cluster 6: European Football

Naming several prominent European Football clubs, 'Chelsea', 'Arsenal', 'Liverpool' and 'Manchester', this section is about the European Football League

Cluster 7: Davis Cup Tennis

The final cluster is about the Davis Cup in tennis. This can be shown by the terms 'Davis Cup' and the names of famous professional tennis players such as 'Roddick', 'Nadal' and 'Moya'.

### 8. Identify the first fifteen high frequent terms (that are not stop words or noise) in the start list?

To find the first fifteen most frequent terms, the remaining terms used in the singular value decomposition are extracted from the complete list of terms ordered by term frequency, created in question 4.

In [ ]:
svdTerms = []
for term in orderedTerms:
    if term['term'] in tfidf_vec.get_feature_names():
        svdTerms.append(term)

In [ ]:
for i, term in enumerate(svdTerms[0:40]):
    print(str(i + 1) + ")\tTerm:", term['term'], "\tFrequency:", term['tf'])

The top fifteen terms are:

1. England
2. Team
3. Match
4. Second
5. Set
6. Give
7. Day
8. Open
9. Final
10. New
11. Want
12. Cup
13. Good
14. Think
15. Club




### 9. Describe how these clusters can be useful in the online personalised news story service planned.

From the clusters discerned in this section, there are several observations that can be made. However, the news articles focus primarily on sport. This may be a result of a biased sample, choosing only sports related articles, but this does not mean the observations are invalid. Instead, these observations can be applied specifically to sports journalism.

Firstly, the clusters seem to be evenly split between different sports. Cricket, tennis, rugby union, football (soccer), and athletics are represented. This implies an even coverage of popular sports is necessary. There may be available audiences for each discipline, and focusing too hard on a singular sport will shrink the audience.

Secondly, the clusters focus on the 'highest' levels of the sport. The Australian Open and Davis Cup for tennis, the international tests for cricket, and the European football leagues. The implication is that there is the most interest around the highest levels and greatest competitions of the sport.

Finally, there are also clusters relating to background machinations and controversy. For example, the drug bans in the Athens Olympics and the potential new managers in European Football. This ties in to the second point, as audiences are most likely interested in the events that can affect the outcomes of their sports championships and tournaments.

---
## Part 4: Web Mining<a name="web"></a>





In [ ]:
import pandas as pd
import os

### 1. Sequential Rule Mining

In [ ]:
df = pd.read_csv('Casestudy2-Data-Py/web_log_data.csv')
df.head(10)

Although the provided data was removed unproductive items such as graphics, sound, etc, we can see that Javascript (.js), Cascading Style Sheets(.css) and Icon (.ico) were not completely filterred. In the context of Web usage mining, those files could be considered as non-meaningful requests. Hene, we will remove all these files as below. 

#### a. What is the Rational behind the selected method

In the context of Web usage mining, sequential rule mining helps to discover inter-session patterns such as the presense set of pages followed by another set of pages ordered by the step visitors browsing pages. By using this approach, web owners can take advantages of the rules to predict visting patterns, then make websites layout beter.

#### b. What variables were included in the analysis and what were their roles and measurement level set? Justify your choice.

<table>
    <tr>
        <th>Variable Name</th>
        <th>Role</th>
        <th>Measurement Level</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>request</td>
        <td>Target</td>
        <td>Norminal</td>
        <td>Cleaned request made to a site</td>
    </tr>
    <tr>
        <td>step</td>
        <td>Sequence</td>
        <td>Ordinal</td>
        <td>Ordered request access made by user</td>
    </tr>
    <tr>
        <td>user_id</td>
        <td>ID</td>
        <td>Norminal</td>
        <td>Identifier of user</td>
    </tr>
</table>

#### c. Can you identify data quality issues in order to perform web mining?

Data issues have been discussed in the data preprocessing.

In [ ]:
def strip_ending_slash(doc):
    """
    Fix data inconsistancies by removing trailing slashes from requests
    """
    # Get Request
    request_string = doc['request']
    # Strip slash from request
    request_string = request_string.strip('/')
    # Add slash back to start of request
    request_string = '/'+request_string
    #return cleaned request to list as new attribute
    doc['request'] = request_string
    
    return doc

In [ ]:
df = df.apply(strip_ending_slash, axis=1)

In [ ]:
def get_request_extension(doc):
    request_string = doc['request']
    path, extension = os.path.splitext(request_string)
    doc['extension'] = extension
    return doc

In [ ]:
df = df.apply(get_request_extension, axis=1)

In [ ]:
unproductiveItems = ['.ico', '.js', '.css']

In [ ]:
df = df[~df.extension.isin(unproductiveItems)]

#### d. Discuss the results obtained. Discuss also the applicability of findings of the method.

The dafafame will be sorted by step

In [ ]:
df = df[['request', 'user_id', 'step']]
df = df.sort_values(by=['step'])

The dafafame will be groupued by user to get a list of requests

In [ ]:
df = df.groupby(['user_id'])['request'].apply(list)

We removed all the lists that have only one item or one request as it would not help in finding visitting patterns.

In [ ]:
sequences = df.values.tolist()

sequences = [a for a in sequences if len(a) != 1]
# show the first 5 sequences
print(sequences[:5])

In [ ]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', '10%', '10%'], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [ ]:
get_association_rules(sequences, 0.1, 0.1).sort_values(by='Confidence', ascending=False)

The first rule is '/' => '/services.html' for 0.18 support and 0.85 confidence. This is a strong rule, as it implies 18% of visitor visited 'eaglefarm/pdf/Web_Price_List.pdf'  after '/' (support) and if visitor visited '/', the probability of them visiting 'eaglefarm/pdf/Web_Price_List.pdf' after is 85%.

---
### 2. Graph Analysis
#### a. What is the Rational behind the selected method
By performing graph analysis on the web logs, it is possible to visualise the structure and useage of the web page.

In [2]:
# Libraries required for this section
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

# Load Data for Web log minig
df = pd.read_csv('Casestudy2-Data-Py/web_log_data.csv')
df.head(10)

,ip,date_time,request,step,session,user_id
0,c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3
1,visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12
2,dsl-61-95-54-84.requ,19/Apr/2005:08:33:01,/,1,13,13
3,d220-236-91-52.dsl.n,19/Apr/2005:09:16:06,/,1,15,15
4,allptrs.eq.edu.au,19/Apr/2005:09:47:54,/,1,22,22
5,cpe-144-136-135-38.q,19/Apr/2005:10:13:37,/,1,23,23
6,225-145-222-203.rev.,19/Apr/2005:11:48:32,/,1,25,25
7,cpe-138-130-198-54.q,19/Apr/2005:12:31:54,/,1,26,26
8,203-219-44-170-qld.t,19/Apr/2005:12:33:49,/,1,29,29
9,cpe-138-130-198-54.q,19/Apr/2005:12:42:51,/,1,30,30


#### b. What variables were included in the analysis and what were their roles and measurement level set? Justify your choice.
<table>
<tr>
<th>Variable</th>
<th>Measurement Level</th>
<th>Role</th>
</tr>
<tr>
<td>Request</td>
<td>Nominal</td>
<td>Node</td>
</tr>
<tr>
<td>Step</td>
<td>Ordinal</td>
<td>Used to identify the edge</td>
</tr>
<tr>
<td>User_ID</td>
<td>Nominal</td>
<td>Used to group steps and requrests</td>
</tr>
</table>

#### c. Can you identify data quality issues in order to perform web mining?
As mentioned in Sequential Rule Mining Section, 

In [4]:
def strip_ending_slash(doc):
    """
    Fix data inconsistancies by removing trailing slashes from requests
    """
    # Get Request
    request_string = doc['request']
    # Strip slash from request
    request_string = request_string.strip('/')
    # Add slash back to start of request
    request_string = '/'+request_string
    #return cleaned request to list as new attribute
    doc['request'] = request_string
    
    return doc

In [5]:
df1 = df.copy()

df1 = df1.apply(strip_ending_slash, axis=1)

In [6]:
print(df1.info())
print("Length Before: ",len(df.request.unique()))
print("Length After: ",len(df1.request.unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5866 entries, 0 to 5865
Data columns (total 6 columns):
ip           5866 non-null object
date_time    5866 non-null object
request      5866 non-null object
step         5866 non-null int64
session      5866 non-null int64
user_id      5866 non-null int64
dtypes: int64(3), object(3)
memory usage: 275.0+ KB
None
Length Before:  114
Length After:  96


In [ ]:
def make_lower(doc):
    """
    Force requests to lower case to reduce unique values
    """
    # Get Request from data
    reqStr = doc['request']
    # Cast string to lower case
    reqStr = reqStr.lower()
    # Add back to Dataframe as new attribute
    doc['request'] = reqStr
    
    return doc


df2 = df1.apply(make_lower,axis=1)

In [ ]:
print("before: ",len(df1.request.unique()))
print("after: ",len(df2.request.unique()))

In [ ]:
web_log = df2.copy()
web_log = web_log[['ip','request','step','user_id']]
web_log.info()

In [ ]:
print(web_log.groupby(['user_id','request','ip'])['step'].value_counts())

# print(web_log.groupby(['user_id'])['request'].value_counts())

#### d. Discuss the results obtained. Discuss also the applicability of findings of the method.


In [ ]:
def buildGraph(doc):
    """
    Build a graphs based on requests relating to an ip
    
    for each unique ip create a node
    for each request for that ip create a node and edge
    """
    print("generating graph...")
    
    # Begin Graph
    G = nx.Graph()
    
    # Group Elements by user and get a list of unique users to iterate through
    sessions = doc.groupby(['user_id'])
    users = doc['user_id'].unique()
    
    for user in users:
        # Get subset of dataframe for each user ordered by step
        
        # Initialise a variable to hold the previous request
        prev_req = ''
        
        session_requests = sessions.get_group(user).sort_values(by=['step'])
        for index, row in session_requests.iterrows():
            #for each step in the session
            req = row['request']
            try:
                # Add Current request to structure
                G.add_node(req)
                
                if prev_req != '':
                    G.add_edge(req,prev_req)
                prev_req = req
            except:
                pass
            
    Nn = G.number_of_nodes()
    Ne = G.number_of_edges()
    print('Complete. there are {} nodes and {} edges in the graph'.format(Nn,Ne))
    
    return G

In [ ]:
def drawGraph(G,nodeList=None, labels=False, layout='spring',nAlpha=0.2,nCol='blue',nSize=600,eAlpha=1, eCol='blue',lSize=8):
    if layout == 'spring':
        pos = nx.spring_layout(G)
    elif layout == 'shell':
        pos = nx.shell_layout(G)
    else:
        pos = nx.spectral_layout(G)
        
    # Draw Nodes
    deg = nx.degree(G)
    
    plt.figure(3,figsize=(20,20))
    
    if nodeList == None:
        nx.draw_networkx_nodes(G,pos, alpha=nAlpha,color=nCol,node_size=[d[1]*nSize for d in deg])#[d[1]**2 for d in deg]
    else:
        nx.draw_networkx_nodes(G,pos,nodelist=nodeLists, alpha=nAlpha,color=nCol,node_size=[d[1]*nSize for d in deg])#[d[1]**2 for d in deg]

    if labels:
        # Draw Labels
        nx.draw_networkx_labels(G,pos,font_size=lSize)
#         nx.draw_networkx_labels(G,pos)
    
    # Draw Edges
    nx.draw_networkx_edges(G,pos,alpha=eAlpha,edge_color=eCol)
    plt.show()

In [ ]:
mask= web_log['request'].value_counts()>30
def shouldTrim(doc):
    drequest = doc['request']
    doc['drop'] = mask[drequest]
    return doc
web_trim = web_log.apply(shouldTrim,axis=1)
web_trim = web_trim[web_trim['drop']==True]

# G2 = buildGraph(web_trimmed)
# drawGraph(G2,nSize=300)

In [ ]:
G = buildGraph(web_trim)
drawGraph(G,nSize=2,eAlpha=0.2,labels=True,layout='spring')
drawGraph(G,nSize=2,eAlpha=0.2,labels=True,layout='shell')
drawGraph(G,nSize=2,eAlpha=0.2,labels=True)

In [ ]:
G = buildGraph(web_trim)
drawGraph(G,nSize=2,eAlpha=0.2,labels=True,layout='spring')
drawGraph(G,nSize=2,eAlpha=0.2,labels=True,layout='shell')
drawGraph(G,nSize=2,eAlpha=0.2,labels=True)

In [ ]:
# Import library for centrality analysis
import community

def centrality(G, Nimportant=5, method='degree'):
    """
        returns a subgraph of G with the most important nodes according to the centrality algorithm chosen
    """
    
    if method == 'eigenvector':
        ranking = nx.eigenvector_centrality_numpy(G)
    elif method == 'degree':
        dn = nx.degree(G)
        degs = [dn[u] for u, i in dn]; degs.sort(); degs = degs[::-1]; degs = degs[:Nimportant]
        important_nodes = [u for u, i in dn if dn[u] in degs]
        Gt=G.subgraph(important_nodes)
        important_nodes = [(u,dn[u]) for u,i in dn if dn[u] in degs]
    else:
        phi = (1+math.sqrt(5))/2.0 # largest eigenvalue of adj matrix
        ranking = nx.katz_centrality_numpy(G,1/phi)
    if method != 'degree':
        important_nodes = sorted(ranking.items(), key=operator.itemgetter(1))[::-1][0:Nimportant]
        dnodes=[n[0] for n in important_nodes]
        Gt = G.subgraph(dnodes)
    with open('Centrality_details.txt','w') as f:
        for var in important_nodes:
            f.write('%s\t %.3f\n' %(var[0],var[1]))
    
    return Gt

Gt = centrality(G, Nimportant=10, method='degree')
drawGraph(Gt, labels=True,lSize=20)

In [ ]:
import community
import warnings, numpy as np

''' Find communities in a social media graph G '''
def communities(G):
    # close all plots where possible
    try:
        plt.clf(); plt.cla(); plt.close()
    except:
        pass
    
    # find best partition
    part = community.best_partition(G)
    values = [part.get(node) for node in G.nodes()]  # get labels
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        nx.draw_spring(G, cmap = plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)
        plt.savefig('Communities.png')
        plt.show()
    
    # measure number of communities and network modularity
    mod, k = community.modularity(part,G), len(set(part.values()))
    print("Number of Communities = %d\nNetwork modularity = %.2f" %(k,mod)) # https://en.wikipedia.org/wiki/Modularity_%28networks%29
    
    # get members of each partition
    part_members, part_len, dCommunity = {}, {}, []
    for usr,par in part.items():
        if par not in part_members.keys():
            part_members[par]=[usr]; part_len[par]=1
        else:
            part_members[par]=part_members[par]+[usr]; part_len[par]+=1
    
    # save community details into text file
    with open('Community_details.txt','w') as f:
        for com in part_len:
            dCommunity.append((com, part_len[com], set(part_members[com])))
            f.write('community %d\t N=%d\t users= ' %(com,part_len[com]))
            f.write('%s\n' %(' '.join(part_members[com])))
    
    dCommunity.sort(key=lambda tup: tup[1])
    return dCommunity[::-1]

com = communities(G)